In [1]:
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas
from reportlab.lib.utils import simpleSplit
from io import BytesIO
import requests
import openai
import time

# Step 1: Configure OpenAI API Key
openai.api_key = ""

# Step 2: Retrieve Location Details
def get_location_details(location):
    url = "https://nominatim.openstreetmap.org/search"
    params = {
        "q": location,
        "format": "json",
        "addressdetails": 1,
    }
    headers = {"User-Agent": "Googlemaps/1.0 (Hello123@example.com)"}

    for attempt in range(3):  # Retry logic
        try:
            response = requests.get(url, params=params, headers=headers, timeout=10)
            if response.status_code == 200:
                data = response.json()
                if data:
                    print("Raw location data retrieved from API:")
                    print(data[0])  # Print first result for debugging
                    return data[0]
                else:
                    print("No results found for the location.")
                    return None
            else:
                print(f"API error: {response.status_code}, {response.text}")
        except requests.exceptions.RequestException as e:
            print(f"Attempt {attempt + 1} failed: {e}")
            time.sleep(2)  # Wait before retrying
    raise Exception("Failed to retrieve location details after multiple attempts.")

# Step 3: Summarize Data Using GPT
def summarize_with_gpt(location_data):
    raw_data = f"Location details: {location_data}"
    prompt = f"Summarize the following location information for a user-friendly report:\n\n{raw_data}"

    for attempt in range(3):  # Retry logic
        try:
            response = openai.ChatCompletion.create(
                model="gpt-3.5-turbo",
                messages=[
                    {"role": "system", "content": "You are a helpful assistant summarizing data."},
                    {"role": "user", "content": prompt},
                ],
                timeout=20  # Increase timeout for OpenAI API
            )
            return response["choices"][0]["message"]["content"]
        except openai.error.APIConnectionError as e:
            print(f"Attempt {attempt + 1} failed: Error communicating with OpenAI: {e}")
            time.sleep(2)
        except Exception as e:
            raise Exception(f"GPT API error: {e}")
    raise Exception("Failed to connect to GPT API after multiple attempts.")

# Step 4: Suggest Similar Places Using GPT
def suggest_similar_places_with_gpt(location_data):
    prompt = f"""
    Based on the following location details, suggest up to 3 similar places. Consider type, relevance, and proximity to suggest locations nearby that users might also enjoy:

    {location_data}

    Please provide the suggestions in a concise bullet-point format.
    """

    for attempt in range(3):  # Retry logic
        try:
            response = openai.ChatCompletion.create(
                model="gpt-3.5-turbo",
                messages=[
                    {"role": "system", "content": "You are a helpful assistant suggesting similar places."},
                    {"role": "user", "content": prompt},
                ],
                timeout=20
            )
            return response["choices"][0]["message"]["content"]
        except openai.error.APIConnectionError as e:
            print(f"Attempt {attempt + 1} failed: {e}")
            time.sleep(2)
        except Exception as e:
            raise Exception(f"GPT API error: {e}")
    raise Exception("Failed to connect to GPT API for similar places.")

# Step 5: Generate Reviews Using GPT (New Function)
def generate_reviews_with_gpt(location_name):
    prompt = f"Write a collection of user reviews for the location: {location_name}. Include both positive and constructive feedback in a few sentences."

    for attempt in range(3):  # Retry logic
        try:
            response = openai.ChatCompletion.create(
                model="gpt-3.5-turbo",
                messages=[
                    {"role": "system", "content": "You are a helpful assistant writing user reviews."},
                    {"role": "user", "content": prompt},
                ],
                timeout=20
            )
            return response["choices"][0]["message"]["content"]
        except openai.error.APIConnectionError as e:
            print(f"Attempt {attempt + 1} failed: {e}")
            time.sleep(2)
        except Exception as e:
            raise Exception(f"GPT API error: {e}")
    raise Exception("Failed to connect to GPT API for reviews.")

# Step 6: Generate PDF Report
def generate_pdf(summary, location, reviews, maps_url, similar_places):
    buffer = BytesIO()
    c = canvas.Canvas(buffer, pagesize=letter)
    width, height = letter

    # Section Title Helper
    def add_section_title(text, y_position):
        c.setFont("Helvetica-Bold", 16)
        c.drawString(50, y_position, text)
        c.line(50, y_position - 5, width - 50, y_position - 5)  # Add a divider line
        return y_position - 20

    # Text Helper
    def add_wrapped_text(text, y_position, font_size=12, max_width=500, line_spacing=14):
        c.setFont("Helvetica", font_size)
        lines = simpleSplit(text, "Helvetica", font_size, max_width)
        for line in lines:
            if y_position < 50:  # Avoid writing beyond the bottom margin
                c.showPage()  # Create a new page
                y_position = height - 50
            c.drawString(50, y_position, line)
            y_position -= line_spacing
        return y_position

    # Title and Subtitle
    c.setFont("Helvetica-Bold", 20)
    c.drawString(50, height - 50, f"Explore {location}")
    c.setFont("Helvetica-Oblique", 14)
    c.drawString(50, height - 80, "A Comprehensive Location Summary and Reviews")
    y_position = height - 120

    # Add Google Maps Link Section
    y_position = add_section_title("Google Maps Link:", y_position)
    c.setFont("Helvetica", 12)
    c.setFillColorRGB(0, 0, 1)  # Hyperlink color
    c.drawString(50, y_position, maps_url)
    c.setFillColorRGB(0, 0, 0)  # Reset to black
    y_position -= 20

    # Add Summary Section
    y_position = add_section_title("Summary:", y_position)
    y_position = add_wrapped_text(summary, y_position)

    # Add Reviews Section
    y_position = add_section_title("Traveler Reviews:", y_position - 20)
    y_position = add_wrapped_text(reviews, y_position)

    # Add Similar Places Section
    y_position = add_section_title("Similar Places:", y_position - 20)
    y_position = add_wrapped_text(similar_places, y_position)

    # Footer
    c.setFont("Helvetica-Oblique", 10)
    c.drawString(50, 30, "Generated by Your Location Explorer")
    c.drawRightString(width - 50, 30, "Page 1")  # Add page number
    c.save()
    buffer.seek(0)
    return buffer

# Step 7: Main Program Logic
def main():
    print("Welcome! Enter a location to get its details summarized in a professional PDF report.")
    location = input("Enter location: ")

    try:
        # Step 1: Get Location Details
        location_details = get_location_details(location)
        if location_details:
            print("Location details retrieved successfully!")

            # Step 2: Summarize Details
            summary = summarize_with_gpt(location_details)
            print("Summary created!")

            # Step 3: Generate Reviews
            reviews = generate_reviews_with_gpt(location)
            print("Reviews created!")

            # Step 4: Generate Google Maps URL
            lat, lon = location_details['lat'], location_details['lon']
            maps_url = f"https://www.google.com/maps?q={lat},{lon}"

            # Step 5: Suggest Similar Places
            similar_places = suggest_similar_places_with_gpt(location_details)
            print("Similar places suggested!")

            # Step 6: Generate PDF
            pdf = generate_pdf(summary, location, reviews, maps_url, similar_places)
            pdf_filename = f"{location.replace(' ', '_')}_summary.pdf"
            with open(pdf_filename, "wb") as f:
                f.write(pdf.read())
            print(f"PDF generated successfully! File saved as {pdf_filename}")
        else:
            print("No details found for the specified location.")
    except Exception as e:
        print(f"Error: {e}")

if __name__ == "__main__":
    main()


Welcome! Enter a location to get its details summarized in a professional PDF report.


Enter location:  Paris


Raw location data retrieved from API:
{'place_id': 89081766, 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. http://osm.org/copyright', 'osm_type': 'relation', 'osm_id': 7444, 'lat': '48.8588897', 'lon': '2.3200410217200766', 'class': 'boundary', 'type': 'administrative', 'place_rank': 15, 'importance': 0.8845663630228834, 'addresstype': 'suburb', 'name': 'Paris', 'display_name': 'Paris, Île-de-France, France métropolitaine, France', 'address': {'suburb': 'Paris', 'city_district': 'Paris', 'city': 'Paris', 'ISO3166-2-lvl6': 'FR-75C', 'state': 'Île-de-France', 'ISO3166-2-lvl4': 'FR-IDF', 'region': 'France métropolitaine', 'country': 'France', 'country_code': 'fr'}, 'boundingbox': ['48.8155755', '48.9021560', '2.2241220', '2.4697602']}
Location details retrieved successfully!
Summary created!
Reviews created!
Similar places suggested!
PDF generated successfully! File saved as Paris_summary.pdf
